# WorldCereal product download

This example illustrates the use of openEO for combining and downloading data from the [ESA WorldCereal](https://esa-worldcereal.org) project.

This project provides a global map of cereals for 2021 at 10m resolution! It can be used as an important base layer for agriculture use cases. Combined with the power of openEO, you can easily generate agricultural statistics over an area of interest, or use this data as a masking layer in an advanced workflow.

In this example, we'll illustrate a fairly simple case of combining two collections into a single image file.

In [1]:
import openeo
c=openeo.connect("openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


We list the available collection id's to make sure we have the right name.

In [2]:
[col['id'] for col in c.list_collections() if "WORLDCEREAL" in col['id']]

['ESA_WORLDCEREAL_ACTIVECROPLAND',
 'ESA_WORLDCEREAL_IRRIGATION',
 'ESA_WORLDCEREAL_TEMPORARYCROPS',
 'ESA_WORLDCEREAL_WINTERCEREALS',
 'ESA_WORLDCEREAL_MAIZE',
 'ESA_WORLDCEREAL_SPRINGCEREALS ']

In [3]:
c.describe_collection("ESA_WORLDCEREAL_MAIZE")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['classification', 'confidence']},
  't': {'extent': ['2020-09-12T00:00:00Z', '2021-12-20T00:00:00Z'],
   'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180.00446428571428, 179.9955357],
   'reference_system': 4326,
   'step': 8.3333333333e-05,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-59.9955357, 80.0044643],
   'reference_system': 4326,
   'step': 8.3333333333e-05,
   'type': 'spatial'}},
 'description': 'WorldCereal maize, globally, at 10m resolution in specific seasons throughout the year. \n\n ##Seasons\n\n The date of observations in this collection is the start date of the season. This start date varies depending on the agro-ecological zone. WorldCereal defines these seasons:\n * wintercereals\n* maize main \n* maize second\n ',
 'experimental': True,
 'extent': {'spatial': {'bbox': [[-180.00446428571428,
     -59.9955357,
     179.9955357,
     80.0044643]]},
  'temporal': {'interval': [['2020-09-12T00:00:00Z',
     '2021-12-20T00:00:00Z']]}},
 'id': 'ESA_WORLDCEREAL_MAIZE',
 'item_assets': {'classification': {'eo:bands': [{'description': 'classification band',
     'name': 'classification'},
    {'description': 'confidence band', 'name': 'confidence'}],
   'gsd': 10,
   'proj:epsg': 4326,
   'raster:bands': [{'classification:classes': [{'description': 'pixel not marked as maize',
       'name': 'not cropland',
       'value': 0},
      {'description': 'classified as maize in the respective season',
       'name': 'cropland',
       'value': 100}]},
    {}],
   'title': 'classification tiff',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized'}},
 'license': 'CC-BY-4.0',
 'links': [{'href': 'https://esa-worldcereal.org',
   'rel': 'alternate',
   'title': 'Product web page.'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections',
   'rel': 'root'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections',
   'rel': 'parent'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections/ESA_WORLDCEREAL_MAIZE',
   'rel': 'self'}],
 'providers': [{'name': 'ESA',
   'roles': ['licensor'],
   'url': 'https://esa-worldcereal.org'},
  {'description': 'VITO remote sensing lead the consortium that produced the WorldCereal product on behalf of ESA. VITO also hosts the resulting archive.',
   'name': 'VITO',
   'processing:level': 'L4',
   'processing:software': {'WorldCereal': '1.0'},
   'roles': ['producer', 'host']}],
 'stac_extensions': ['https://stac-extensions.github.io/datacube/v2.2.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.1.0/schema.json'],
 'stac_version': '0.9.0',
 'summaries': {'federation:backends': ['vito']},
 'title': 'ESA WorldCereal maize'}

In the following block, we combine two WorldCereal collections into a single output. 
The formula used here is just an example, and can be made much more complex depending on your use case.

In [4]:
extent = {'west': 3.0, 'south': 50.0, 'east': 4.0, 'north': 51.0, 'crs': 'EPSG:4326'}
temporal = ('2020-09-01T00:00:00Z', '2021-12-31T00:00:00Z')
maize = c.load_collection("ESA_WORLDCEREAL_MAIZE",
                         temporal_extent= temporal,
                         spatial_extent=extent,
                         bands=["classification"])
winter = c.load_collection("ESA_WORLDCEREAL_WINTERCEREALS",
                         temporal_extent= temporal,
                         spatial_extent=extent,
                         bands=["classification"]).apply(lambda x:100*(x+10))

combined = maize.merge_cubes(winter, overlap_resolver="sum")


We now have defined what openEO calls a 'process graph', but still need to execute it.
We will use an 'asynchronous' [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html#batch-jobs) that gets sent to the server, as it can take a longer time to execute.

When finished, this command will automatically download the result from openEO to your local working directory.
You can also follow the progress and view results in the [openEO web editor](https://editor.openeo.cloud).

In [ ]:
job = combined.execute_batch("worldcereal_combined.nc")

In [ ]:
job.get_results()